In [47]:
import pandas as pd

In [48]:
jan_data = pd.read_parquet('fhv_tripdata_2021-01.parquet')
jan_data.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [49]:
feb_data = pd.read_parquet('fhv_tripdata_2021-02.parquet')
feb_data.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


# Q1

In [50]:
len(jan_data)

1154112

# Q2

In [51]:
jan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154112 entries, 0 to 1154111
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1154112 non-null  object        
 1   pickup_datetime         1154112 non-null  datetime64[ns]
 2   dropOff_datetime        1154112 non-null  datetime64[ns]
 3   PUlocationID            195845 non-null   float64       
 4   DOlocationID            991892 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1153227 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 61.6+ MB


In [52]:
jan_data['duration']=jan_data['dropOff_datetime']-jan_data['pickup_datetime']

In [53]:
jan_data['duration'].mean()

Timedelta('0 days 00:19:10.033445627')

# Q3

In [54]:
from datetime import timedelta

In [55]:
jan_data['duration_seconds']=jan_data['duration']/timedelta(seconds=1)

In [56]:
jan_data=jan_data.loc[(jan_data['duration_seconds']<=3600)&(jan_data['duration_seconds']>=60),:]/60

In [57]:
jan_data.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,duration_seconds
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,0 days 00:17:00,1020.0
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,0 days 00:17:00,1020.0
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,0 days 00:08:17,497.0
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,0 days 00:15:13,913.0
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,0 days 00:09:03,543.0


In [58]:
jan_data[['PUlocationID','DOlocationID']] = jan_data[['PUlocationID','DOlocationID']].fillna(-1)

# Q4

In [59]:
jan_data[['PUlocationID','DOlocationID']]=jan_data[['PUlocationID','DOlocationID']].astype(str)

In [40]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [60]:
train_dicts = jan_data[['PUlocationID','DOlocationID']].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Q5

In [71]:
y_train = jan_data['duration_seconds'].values

In [72]:
lr = LinearRegression()

In [73]:
lr.fit(X_train,y_train)

LinearRegression()

In [74]:
y_pred = lr.predict(X_train)


In [90]:
mean_squared_error(y_train,y_pred,squared=False)

10.52851910721158

# Q6

In [76]:
feb_data['duration']=feb_data['dropOff_datetime']-feb_data['pickup_datetime']

In [77]:
feb_data['duration_seconds']=feb_data['duration']/timedelta(seconds=1)

In [79]:
feb_data=feb_data.loc[(feb_data['duration_seconds']<=3600)&(feb_data['duration_seconds']>=60),:]

In [81]:
feb_data['duration_seconds']=feb_data['duration_seconds']/60

In [82]:
feb_data[['PUlocationID','DOlocationID']] = feb_data[['PUlocationID','DOlocationID']].fillna(-1)

In [83]:
feb_data[['PUlocationID','DOlocationID']]=feb_data[['PUlocationID','DOlocationID']].astype(str)

In [84]:
val_dicts = feb_data[['PUlocationID','DOlocationID']].to_dict(orient='records')

val_dv = DictVectorizer()
X_val = val_dv.fit_transform(val_dicts)

In [86]:
X_val[:,:-1]

<990113x525 sparse matrix of type '<class 'numpy.float64'>'
	with 1980221 stored elements in Compressed Sparse Row format>

In [88]:
y_val_pred=lr.predict(X_val[:,:-1])

In [91]:
y_val = feb_data['duration_seconds'].values

In [92]:
mean_squared_error(y_val,y_val_pred,squared=False)

18.35639398058831